In [58]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
Prepare consolidated data

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json

print(__doc__)

def read(file):
    data = pd.read_excel(file)
    return data

def clean_data(data):
    
    # Select numeric data alone from data object
    numeric_data = data.loc[:, data.dtypes != object]
    
    # Select string data alone from data object to trim
    trim_data = data.select_dtypes(['object'])
    
    # Trim all string objects
    trim_data = trim_data.apply(lambda x: x.str.strip())
    
    #Merge numeric and string data in to data object
    data = pd.concat([numeric_data, trim_data], axis=1)
    
    # Change column name to lower case
    data.columns = [x.lower() for x in data.columns]

    return data

def get_data():

    # Read customer details excel
    data1 = read("/home/azureuser/Aerospike/data/customer/customer details.xls")
    
    # Clean data1
    data1 = clean_data(data1)

    # Read customer address excel
    data2 = read("/home/azureuser/Aerospike/data/customer/customer_address.xls")
    
    # Clean data2
    data2 = clean_data(data2)
     
    grouped = data2.groupby('cust_id')
    print (grouped)
    
    from collections import defaultdict
    
    results = defaultdict(lambda: defaultdict(dict))
        
    j = (data2.groupby(['cust_id'], as_index=True)
         .apply(lambda x: x[['address_type', 'strt_address', 'city', 'state', 'country', 'pincode']].to_dict('r'))
         .reset_index()
         .rename(columns={0:'address'})
         .to_json(orient='records'))
    
    j1 = json.dumps(json.loads(j), indent=2, sort_keys=True)
    key_data2 = data2.drop_duplicates('cust_id').sort_values('cust_id')
    print (key_data2['cust_id'])
    
    
if __name__ == '__main__':
    get_data()


Prepare consolidated data


37       44261559
2        84711641
49      208312345
63      217237812
14      242339097
74      682216554
88      956851195
68     1080278058
67     1129294040
40     1176389539
27     1213351171
24     1340761503
71     1434949636
3      1561684799
61     1591805119
66     1595285593
73     1788688153
26     1804203393
7      1907459679
6      2006085121
17     2009007839
36     2061945712
25     2071190194
12     2093269256
44     2094139645
77     2157335495
64     2169012813
78     2216013972
100    2300372819
0      2396305636
          ...    
29     6784656776
81     7213476474
79     7275477859
84     7485557938
76     7550054702
33     7945598145
18     8126984299
62     8228865449
83     8349364525
9      8486613847
21     8529245634
41     8782507386
11     8833668145
98     8928475822
22     8969745570
45     8992995570
4      9006719754
97     9112386251
48     9129126383
90     9329017447
80     9419697469
32     9582619573
28     9601344591